# **Data Collection**

## Objectives

* Fetch data from Kaggle and save as raw data
* Check for non-image files
* Split into Test, Train and Validation sets

## Inputs

* Kaggle JSON file - authentication token

## Outputs

Generate Dataset: inputs/cherry-leaf-dataset/cherry-leaves

## Additional Comments

* This notebook imports, cleans and prepares the data for the machine learning model.



---

# Change working directory

* The notebooks are stored in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/cherry-leaf-mildew-detection-project/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/cherry-leaf-mildew-detection-project'

## Install Packages

In [4]:
## Install requirements to avoid issues with joblib import
%pip install -r requirements.txt

  Using cached numpy-1.19.2-cp38-cp38-manylinux2010_x86_64.whl (14.5 MB)
  Using cached pandas-1.1.2-cp38-cp38-manylinux1_x86_64.whl (10.4 MB)
  Using cached matplotlib-3.3.1-cp38-cp38-manylinux1_x86_64.whl (11.6 MB)
  Using cached seaborn-0.11.0-py3-none-any.whl (283 kB)
  Using cached plotly-4.12.0-py2.py3-none-any.whl (13.1 MB)
  Using cached streamlit-0.85.0-py2.py3-none-any.whl (7.9 MB)
  Using cached scikit_learn-0.24.2-cp38-cp38-manylinux2010_x86_64.whl (24.9 MB)
  Using cached tensorflow_cpu-2.6.0-cp38-cp38-manylinux2010_x86_64.whl (172.4 MB)
  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)
  Using cached protobuf-3.20.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Using cached altair-4.2.2-py3-none-any.whl (813 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Using cached kiwisolver-1.4.5-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.2 MB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached pyparsing-3.1.2-py

---

# Install Kaggle

Install Kaggle so the dataset can be imported.

In [5]:
# Install Kaggle Package
%pip install kaggle==1.5.12


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Run the cell below to change the Kaggle configuration directory to the current working directory and set permissions for the Kaggle authentication JSON.

In [6]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Import the data from Kaggle

In [7]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry-leaf-dataset"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 91%|██████████████████████████████████▌   | 50.0M/55.0M [00:02<00:00, 29.6MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:02<00:00, 27.0MB/s]


In [8]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# Data Preparation

Check the data for any files which are not images.

In [9]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location)  # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))

Remove any non image files from the dataset

In [10]:
remove_non_image_file(my_data_dir='inputs/cherry-leaf-dataset/cherry-leaves')

Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0
Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0


---

# Split Data into Train, Test, Validation Sets

Split the data into train (70%, 0.7), test (20%, 0.2) and vaildation (10%, 0.1) sets.

In [11]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)  # it should get only the folder name
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)

In [12]:
split_train_validation_test_images(my_data_dir=f"inputs/cherry-leaf-dataset/cherry-leaves",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )

---

# Push files to Repo

* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [13]:
#import os
#try:
#    # create here your folder
#    # os.makedirs(name='')
#except Exception as e:
#    print(e)
